In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report, roc_auc_score
warnings.filterwarnings('ignore')

## Wczytanie i prygotowanie danych

Dane podzieliłem na 3 zbiory: treningowy, testowy i walidacyjny.

In [4]:
df = pd.read_csv('australia.csv')

In [5]:
df.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [6]:
x = df.drop('RainTomorrow', axis = 1)
y = df.RainTomorrow
x_train, x_test, y_train, y_test = train_test_split(x, y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test)

## Trenowanie modeli

Postanowiłem sprawdzić jak działać będzie:
- pojedyńcze drzewo w którym narzuciłem minimalną liczbę danych w liściu na 300 aby zapobiec przeuczaniu się modelu
- regresja logistyczna której zmieniłem karę za liczbę zminnych, ponieważ i tak mamy dużo obserwacji
- las losowy, sprawdzę czy faktycznie jest lepszy od pojedyńczego drzewa, zwiększyłem tu liczbę dzrzew do 150 oraz minimalą wartośCI w liściu DO 300

In [34]:
tree = DecisionTreeClassifier(min_samples_leaf=300)
tree.fit(x_train, y_train)
tree.score(x_test, y_test)

0.8483645301569295

In [31]:
lr = LogisticRegression(penalty='none', max_iter=500)
lr.fit(x_train, y_train)
lr.score(x_test, y_test)

0.8575345055776139

In [15]:
rf = RandomForestClassifier(n_estimators=150, min_samples_leaf=300)
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

0.8548875023633957

## Badanie skuteczności

Aby zbadać skuteczność modeli sprawdziłem metodą średniej z crossvalidacji każdy z nich dla 4 równych metryk. Ogólne accuracy jest podobne dla każdego z modeli, gdzie Regresja Logistyczna jest o ok 0.5 pp lepsza.   
Aby upewnić się co do wyniku sprawdzam modele na zbiorze testowym. W tym wypadku róznica w Accuracy jest jeszcze mniejsza. Warto się zastanownić czy któraś z miar nie jest dla nas ważniejsza. Uważam, że wyłapywanie dni które są deszczowe jest dla nas jak najbardziej kluczowe, dlatego powinniśmy dużą uwagę przyłożyć do miary Recall. W tym wypadku Log Reg jest dużo lepsze od pozostałych modeli, choć nadal z dość słabym wynikiem, nieco ponad 52%.  

Biorąc pod uwagę ponniższe wyniki uważam, że Log Reg jest najlepszym modelem do naszego problemu.

In [18]:
df = pd.DataFrame(columns=['accuracy','precision','recall', 'roc_auc'], index=['Tree', 'Log Reg', 'Random Forest'])
ls = [tree, lr, rf]
for i in range(0, len(ls)):
    for metric in df.columns.values:
        df.loc[df.index[i],metric] = np.mean(cross_val_score(ls[i], x_train, y_train, scoring=metric))
df

,accuracy,precision,recall,roc_auc
Tree,0.845374,0.719639,0.496483,0.866734
Log Reg,0.851826,0.730565,0.525895,0.880104
Random Forest,0.849321,0.761191,0.467711,0.881172


In [28]:
tree_y_hat = tree.predict(x_test)
lr_y_hat = lr.predict(x_test)
rf_y_hat = rf.predict(x_test)
ls_hat = [tree_y_hat, lr_y_hat, rf_y_hat]

In [43]:
df1 = pd.DataFrame(columns=['accuracy','precision','recall', 'roc_auc'], index=['Tree', 'Log Reg', 'Random Forest'])
metric = df1.columns.values
for i in range(0, len(ls)):
    df1.loc[df.index[i],metric[0]] = accuracy_score(y_test, ls_hat[i])
    df1.loc[df.index[i],metric[1]] = precision_score(y_test, ls_hat[i])
    df1.loc[df.index[i],metric[2]] = recall_score(y_test, ls_hat[i])
    df1.loc[df.index[i],metric[3]] = roc_auc_score(y_test, ls_hat[i])

In [45]:
df1

,accuracy,precision,recall,roc_auc
Tree,0.847797,0.699387,0.504425,0.722758
Log Reg,0.857535,0.731123,0.526991,0.737167
Random Forest,0.854888,0.76023,0.468584,0.714215
